# SOLID Principles In Python

SOLID talks about five core principles that intend to make software designs more flexible, understandable and maintainable. These principles are :

1. Single Responsibility Principle

2. Open Closed Principle

3. Liskov Substitution Principle

4. Interface Segregation Principle

5. Dependency Inversion Principle 

## 1. Single Responsibility Principle

A class should have only one responsibility (A class should have only one reason to change.)

The following code violates SRP because the `TransactionManager` class is responsible for both generating transaction details and saving them to a database.

In [ ]:
class TransactionManager:
    def generate_transaction(self):
        # Code to generate transaction details
        return {"account": "123456", "amount": 1000, "type": "credit"}

    def save_transaction(self):
        transaction = self.generate_transaction()
        with db.session() as session:
            session.insert(transaction)

# Client code
tm = TransactionManager()
tm.save_transaction()

**Refactored Example: Following SRP**

We split the responsibilities into smaller classes: one for generating transaction details and another for saving them to the database.

In [ ]:
class TransactionGenerator:
    def generate(self):
        # Code logic to generate transaction details
        return {"account": "123456", "amount": 1000, "type": "credit"}

class TransactionWriter:
    def write(self, transaction):
        with db.session() as session:
            session.insert(transaction)

class TransactionManager:
    def __init__(self, generator: TransactionGenerator, writer: TransactionWriter):
        self._generator = generator
        self._writer = writer

    def save_transaction(self):
        transaction = self._generator.generate()
        self._writer.write(transaction)

# Client code
tm = TransactionManager(TransactionGenerator(), TransactionWriter())
tm.save_transaction()

**Benefits of the Refactored Code**

1. Single Responsibility:
`TransactionGenerator` is responsible for generating transaction details.
`TransactionWriter` is responsible for saving transaction details to the database.

2. Easier Maintenance: Changes to transaction generation or database logic can be made independently.

3. Testability: Each class can be tested in isolation.

This design adheres to the Single Responsibility Principle and ensures the code is modular, maintainable, and extensible.

## 2. Open-Closed Principle (OCP)

Software entities (classes, modules, functions, etc.) should be open for extension, but closed for modification.



The following code violates OCP because the TaxCalculator class needs to be modified every time a new tax type is added. We need to modify both the `__init__` and `calculate_tax` 



In [1]:
class TaxCalculator:
    def __init__(self, tax_type, **kwargs):
        self.tax_type = tax_type
        if self.tax_type == "income_tax":
            self.income = kwargs["income"]
            self.rate = kwargs["rate"]
        elif self.tax_type == "sales_tax":
            self.amount = kwargs["amount"]
            self.rate = kwargs["rate"]

    def calculate_tax(self):
        if self.tax_type == "income_tax":
            return self.income * self.rate
        elif self.tax_type == "sales_tax":
            return self.amount * self.rate

# Example Usage
income_tax = TaxCalculator("income_tax", income=50000, rate=0.2)
print(income_tax.calculate_tax())  # Output: 10000

sales_tax = TaxCalculator("sales_tax", amount=200, rate=0.1)
print(sales_tax.calculate_tax())  # Output: 20

10000.0
20.0


**Refactored Example: Following OCP**

We refactor the code by introducing an abstract base class Tax and creating separate classes for each tax type.



In [2]:
from abc import ABC, abstractmethod

# Abstract base class for Tax
class Tax(ABC):
    @abstractmethod
    def calculate_tax(self):
        pass

# Concrete class for Income Tax
class IncomeTax(Tax):
    def __init__(self, income, rate):
        self.income = income
        self.rate = rate

    def calculate_tax(self):
        return self.income * self.rate

# Concrete class for Sales Tax
class SalesTax(Tax):
    def __init__(self, amount, rate):
        self.amount = amount
        self.rate = rate

    def calculate_tax(self):
        return self.amount * self.rate

# Concrete class for Property Tax
class PropertyTax(Tax):
    def __init__(self, property_value, rate):
        self.property_value = property_value
        self.rate = rate

    def calculate_tax(self):
        return self.property_value * self.rate

# Example Usage
income_tax = IncomeTax(income=50000, rate=0.2)
print(income_tax.calculate_tax())  # Output: 10000

sales_tax = SalesTax(amount=200, rate=0.1)
print(sales_tax.calculate_tax())  # Output: 20

property_tax = PropertyTax(property_value=300000, rate=0.01)
print(property_tax.calculate_tax())  # Output: 3000

10000.0
20.0
3000.0


## 3. Liskov Substitution Principle (LSP)

> Subtypes must be substitutable for their base types.

Let’s say we have a `BankAccount` class that represents a generic bank account. We then create a `SavingsAccount` class as a subclass of `BankAccount`. However, the `SavingsAccount` class overrides the behavior of the `withdraw` method in a way that violates the expectations of the base class.

In [ ]:
class BankAccount:
    def __init__(self, balance):
        self.balance = balance

    def deposit(self, amount):
        self.balance += amount

    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount

class SavingsAccount(BankAccount):
    def __init__(self, balance, withdrawal_limit):
        super().__init__(balance)
        self.withdrawal_limit = withdrawal_limit

    def withdraw(self, amount):
        if amount > self.withdrawal_limit:
            raise ValueError("Withdrawal amount exceeds the limit")
        super().withdraw(amount)

# Example Usage
account = BankAccount(1000)
account.withdraw(500)  # Works fine

savings = SavingsAccount(1000, 300)
savings.withdraw(500)  # Raises ValueError: Withdrawal amount exceeds the limit

**Refactored Example: Following LSP**

To fix this, we can create a base class `Account` with a common interface and make `BankAccount` and `SavingsAccount` siblings rather than having a parent-child relationship.

In [3]:
from abc import ABC, abstractmethod

class Account(ABC):
    def __init__(self, balance):
        self.balance = balance

    @abstractmethod
    def deposit(self, amount):
        pass

    @abstractmethod
    def withdraw(self, amount):
        pass

class BankAccount(Account):
    def deposit(self, amount):
        self.balance += amount

    def withdraw(self, amount):
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount

class SavingsAccount(Account):
    def __init__(self, balance, withdrawal_limit):
        super().__init__(balance)
        self.withdrawal_limit = withdrawal_limit

    def deposit(self, amount):
        self.balance += amount

    def withdraw(self, amount):
        if amount > self.withdrawal_limit:
            raise ValueError("Withdrawal amount exceeds the limit")
        if amount > self.balance:
            raise ValueError("Insufficient funds")
        self.balance -= amount

# Example Usage
def process_withdrawal(account: Account, amount):
    account.withdraw(amount)
    print(f"New balance: {account.balance}")

# Using BankAccount
bank_account = BankAccount(1000)
process_withdrawal(bank_account, 500)  # New balance: 500

# Using SavingsAccount
savings_account = SavingsAccount(1000, 300)
process_withdrawal(savings_account, 200)  # New balance: 800

New balance: 500
New balance: 800


**Benefits**

- **Substitutability**: Both BankAccount and SavingsAccount can be used wherever an Account is expected without breaking the code.

- **Extensibility**: New account types can be added without modifying existing classes.

- **Maintainability**: Each class has a clear and independent responsibility.

## 4. Interface Segregation Principle (ISP)

> No code should not be forced to depend on methods that they do not use.

Let’s say we have a `PaymentProcessor` class that provides methods for processing payments, refunds, and generating invoices. However, not all payment processors support all these functionalities. For example, some processors only handle payments, while others handle payments and refunds but not invoices.


In [ ]:
from abc import ABC, abstractmethod

class PaymentProcessor(ABC):
    @abstractmethod
    def process_payment(self, amount):
        pass

    @abstractmethod
    def process_refund(self, amount):
        pass

    @abstractmethod
    def generate_invoice(self, transaction_id):
        pass

class BasicPaymentProcessor(PaymentProcessor):
    def process_payment(self, amount):
        print(f"Processing payment of ${amount}...")

    def process_refund(self, amount):
        raise NotImplementedError("Refund functionality not supported")

    def generate_invoice(self, transaction_id):
        raise NotImplementedError("Invoice generation not supported")

class AdvancedPaymentProcessor(PaymentProcessor):
    def process_payment(self, amount):
        print(f"Processing payment of ${amount}...")

    def process_refund(self, amount):
        print(f"Processing refund of ${amount}...")

    def generate_invoice(self, transaction_id):
        print(f"Generating invoice for transaction {transaction_id}...")

**Refactored Example: Following ISP**

To fix this, we can segregate the `PaymentProcessor` interface into smaller, more specific interfaces.

In [4]:
from abc import ABC, abstractmethod

# Separate interfaces for specific responsibilities
class Payment(ABC):
    @abstractmethod
    def process_payment(self, amount):
        pass

class Refund(ABC):
    @abstractmethod
    def process_refund(self, amount):
        pass

class Invoice(ABC):
    @abstractmethod
    def generate_invoice(self, transaction_id):
        pass

# BasicPaymentProcessor only implements the Payment interface
class BasicPaymentProcessor(Payment):
    def process_payment(self, amount):
        print(f"Processing payment of ${amount}...")

# AdvancedPaymentProcessor implements all interfaces
class AdvancedPaymentProcessor(Payment, Refund, Invoice):
    def process_payment(self, amount):
        print(f"Processing payment of ${amount}...")

    def process_refund(self, amount):
        print(f"Processing refund of ${amount}...")

    def generate_invoice(self, transaction_id):
        print(f"Generating invoice for transaction {transaction_id}...")

In [5]:
# Using BasicPaymentProcessor
basic_processor = BasicPaymentProcessor()
basic_processor.process_payment(100)  # Output: Processing payment of $100...

# Using AdvancedPaymentProcessor
advanced_processor = AdvancedPaymentProcessor()
advanced_processor.process_payment(200)  # Output: Processing payment of $200...
advanced_processor.process_refund(50)    # Output: Processing refund of $50...
advanced_processor.generate_invoice("TXN12345")  # Output: Generating invoice for transaction TXN12345...

Processing payment of $100...
Processing payment of $200...
Processing refund of $50...
Generating invoice for transaction TXN12345...


**Benefits**

* **No Unused Methods:** Classes only implement the methods they need.

* **Extensibility:** Adding new functionalities (e.g., RecurringPayment) doesn’t affect existing classes.

* **Adherence to ISP:** Each interface has a single responsibility, and clients (classes) depend only on the interfaces they use.

## 5. Dependency Inversion Principle (DIP)

Abstractions should not depend upon details. Details should depend upon abstractions.

Let’s say we have a `TransactionProcessor` class that processes transactions. It directly depends on a Database class to save transaction data. This creates tight coupling between `TransactionProcessor` and `Database`.

In [ ]:
class TransactionProcessor:
    def __init__(self):
        self.database = Database()

    def process_transaction(self, transaction):
        self.database.save_transaction(transaction)

class Database:
    def save_transaction(self, transaction):
        print(f"Saving transaction to the database: {transaction}")

# Example Usage
processor = TransactionProcessor()
processor.process_transaction({"account": "123456", "amount": 1000, "type": "credit"})

**Refactored Example: Following DIP**

To fix this, we introduce an abstraction (`TransactionStorage`) that `TransactionProcessor` depends on. Concrete implementations like `DatabaseStorage` and `FileStorage` will implement this abstraction

In [6]:
from abc import ABC, abstractmethod

# Abstraction for transaction storage
class TransactionStorage(ABC):
    @abstractmethod
    def save_transaction(self, transaction):
        pass

# Concrete implementation for database storage
class DatabaseStorage(TransactionStorage):
    def save_transaction(self, transaction):
        print(f"Saving transaction to the database: {transaction}")

# Concrete implementation for file storage
class FileStorage(TransactionStorage):
    def save_transaction(self, transaction):
        print(f"Saving transaction to a file: {transaction}")

# TransactionProcessor depends on the abstraction, not a concrete implementation
class TransactionProcessor:
    def __init__(self, storage: TransactionStorage):
        self.storage = storage

    def process_transaction(self, transaction):
        self.storage.save_transaction(transaction)

# Example Usage
db_processor = TransactionProcessor(DatabaseStorage())
db_processor.process_transaction({"account": "123456", "amount": 1000, "type": "credit"})
# Output: Saving transaction to the database: {'account': '123456', 'amount': 1000, 'type': 'credit'}

file_processor = TransactionProcessor(FileStorage())
file_processor.process_transaction({"account": "123456", "amount": 1000, "type": "credit"})
# Output: Saving transaction to a file: {'account': '123456', 'amount': 1000, 'type': 'credit'}

Saving transaction to the database: {'account': '123456', 'amount': 1000, 'type': 'credit'}
Saving transaction to a file: {'account': '123456', 'amount': 1000, 'type': 'credit'}


**Benefits**

1. **Flexibility:** You can easily switch between different storage mechanisms (e.g., database, file, API) without modifying TransactionProcessor.

2. **Extensibility:** Adding a new storage mechanism (e.g., cloud storage) only requires creating a new class that implements TransactionStorage.

3. **Testability:** You can mock the TransactionStorage interface for unit testing TransactionProcessor.